## MEMORIA DEL PROYECTO


He comenzado a analizar y estudiar varios Datasets de la página de calificación de cine IMDB, he realizado una limpieza superficial para eliminar variables redundantes o con datos no usables para modelos predictivos. 

Enlace a los datasets brutos. [https://datasets.imdbws.com/]

Con ellos he tabajado con un total de 3 datasets: 

         Title.ratings.tsv

Donde estoy usando las columnas *averageRating* que veo la puntuación promedio de cada obra y *numVotes* para ver el número total de votos en IMDB



        Title.basics.tsv

Donde rescato las columnas *titleType*, *primaryTitle*, *starYear*, *runtimeMinute* y *genres* para ver el tipo de la obra, el título, el género y el año de estreno. También uso *PrimaryName* con el nombre del actor/actriz principal.



        Title.principals.tsv
        
ID del IMDB del actor/actriz protagonista (puede ser redundante)



____

____

### LIMPIEZA Y ORGANIZACIÓN DE LOS DATOS    12/05/2025

El último dataset con el que me quedo tiene un ttal de 323 mil filas y 10 columnas en un principio. Comienzo a ver sus características y y sus punto fuertes y débiles. 

1. Veo que en la columna **RunTime** tengo bastantes nulos, por tanto decido crear un nuevo dataset con sOlamente obras que superen los 60 minutos de duración, por lo que desaparecen esos nulos y el dataset se queda finalmentecon 140 mil filas. 

2. Creo una nueva columna numérica (TitleType_num) a partir de la columna TitleType numérica para poder tratarla más adelante 

3. Comienzo con los primeros gráficos y visualizo un mapa de Correlación, de momento, solamente con las columnas numéricas. 

![Correlacion_var_numéricas](graficos_eda/correlacion_variables_num.png)


    Lo que más llama la atención: runtimeMinutes (-0.364): Correlación negativa moderada.
Hallazgo interesante: A mayor duración, menor puntuación en promedio.

4. Miro también el mapa de correlación pero con el dataset over 60 para ver las diferencias entre ellos en el resultado. 

![Correlacion_var_numéricas](graficos_eda/correlacion_var_over_60.png)

    Aquí vemos cómo se pierde la correlación que existía entre la duración de la obra y su nota media, dando a entender que cuando eliminamos las obras de menos de 60 minutos de duración, la correlación pasa a ser positiva pero débil: 0.166
Esto podría indicar que películas/series con duraciones adecuadas (ni muy cortas ni muy largas) son mejor valoradas.

##### Pairplot

Vamos a ver el pairplot con las columnas numéricas en un primer momento para ver el resultado y a relación entre variables.

![Pairplot](graficos_eda/pairplot.png)

1. averageRating vs. numVotes:  Hay una tendencia débil: ratings más altos suelen tener más votos, pero también hay muchos valores dispersos.
    Esta relación no es lineal clara, pero sí podría ser útil como feature.

2. averageRating vs. startYear o decade: Distribución muy dispersa.
    Podría haber una ligera tendencia temporal, pero no parece determinante por sí sola.

3. averageRating vs. runtimeMinutes:  Prácticamente no hay correlación clara.
    Mucha dispersión, incluso con outliers extremos (películas de 4000 minutos).

4. averageRating vs. titleType_num: Muestra estratificación discreta (ya que es una variable categórica transformada).
    Podría ser útil si se codifica correctamente (OneHot o Embedding).


#### Conclusión

De momento, veo que solamente con las variables numéricas no me sirve para funcionar en un buen modelo predictivo. 

Vamos a ver ahora la distribución de ratings: 

![Distribución_ratings](graficos_eda/distribucion_rating.png)

Donde podemos ver que la maroyía de las puntuaciones se quedan entre el 6,5 y el 8 con una cola poco relevante hacia el 0 y una bajada muy significativa en la puntuación máxima.

En conclusión, las variables numéricas de mi dataset no son robustas y no hay una correlación clara, por tanto, no me vaticina un resultado óptimo. 



____
____

**Decido entonces al no tener un dataset muy robusto, trabajar con las variables no numéricas que tiene.**


1. PRIMERO, Reduzco la cantidad de filas en base a la columna **runMinute**. Elimino directamente las obras menores de 60 minutos ya que quiero centrarme en obras largas y decido así mismo eliminar todos los valores nulos que había en la columna. 


2. SEGUNDO. Elimino todos los nulos en las variables clave para que no interfieran en el análisis. Me quedo así con poco más de 240 mil filas. 

3. TERCERO. Conteo los géneros, creo una columna por cada uno de ellos con un definición binaria y aquellos géneros que aparecen menos de 100 veces, se colocan en una columna aparte llamada otros géneros para no saturar de información irrelevante. 

4. CUARTO. Creo una clumna categórica con el **AverageRating** en la que se clasifican como MALA < 4.5, REGULAR (4.6 - 6,5), BUENA (6,6 - 8,9), MUY BUENA (9 - 10)

**MODIFICACIÓN**


### SIGO GRAFICÁNDO CON EL RESTO DE INFORMACIÓN 


![MEDIA_GENERO](graficos_eda/media_por_genero.png)

El gráfico muestra qué géneros tienden a tener mejores valoraciones promedio

        Géneros con barras más altas → tienen mejores ratings promedio.

        Géneros con barras más bajas → tienden a tener valoraciones más bajas.


 ¿Para qué sirve esto?

1. Ayuda a identificar patrones en el comportamiento del público.

2. Tomar decisiones de negocio (por ejemplo, qué géneros producir o promocionar).

3. Ingeniería de características (usar ls información para enriquecer modelos predictivos).

![Distribución_catg](graficos_eda/distribucion_rating_categ.png)

Vemos como en la categoría de MALA hay bastantes outliers muy bajos con una puntuación realmente baja mientras que el resto muestran bastantes menos. 

Por otro lado, también tenemos que la media esta bastante centrada en la mayoría exceptuando tal vez MUY BUENA que tiende a la baja ya que es muy raro conseguir una puntuación total de 10. 

![Tipos_obras](graficos_eda/tipos_obras.png)

Para mostrar y confirmar que no es muy predictiva por lo sesgado de la información, graficamos la cantidad de tipos de obras que tenemos en el dataset para verlo.



![Decada](graficos_eda/rating_x_decada.png)

Con la variable década ya que el promedio ronda entre los 5 y los 6.5 puntos en todas las décadas prácticamente. 

![rating_titulo](graficos_eda/rating_x_titulo.png)

***

***

#### TOCA AHORA HACER POR FIN EL TRAIN TEST SPLIT Y COMENZAR A TRABAJAR DE LLENO CON LOS MÉTODOS


### Primera toma de contacto con el análisis predictivo: REGRESIÓN.

Vamos a empezar a trabajar con el dataset una vez está procesado y limpiado. Comenzaré con un modelo sencillo de regresión para predecir **Average Rating**.

| Modelo                        | Ventajas                                              | Consideraciones                                  |
|------------------------------|--------------------------------------------------------|--------------------------------------------------|
| Linear Regression / Ridge / Lasso | Rápidos, interpretables                             | Pueden quedarse cortos si hay no linealidad      |
| Random Forest Regressor      | Captura no linealidades y es robusto                  | Puede sobreajustar si no se regula               |
| Gradient Boosting (XGBoost / LightGBM) | Muy preciso, maneja missing y outliers         | Más lento de entrenar, requiere tuning           |
| Neural Network (MLP)         | Ideal si tienes muchas variables, como géneros        | Requiere más datos y tuning                      |




### Analizando modelos con Clasificación

Predeciremos **rating_cat**. eniendo en cuenta que el averaceRating es un derivado directo. 


| Modelo                            | Ventajas                                                 | Consideraciones                              |
|-----------------------------------|----------------------------------------------------------|----------------------------------------------|
| Logistic Regression (multiclase)  | Baseline rápido e interpretable                          | Puede necesitar regularización y no capta relaciones no lineales              |
| Random Forest Classifier          | Bueno para features tabulares y mezclas num/categ        | Puede ser pesado en memoria y lento                     |
| XGBoost / LightGBM Classifier     | Alta precisión y eficiencia en das desvalanceados(mi caso)                            | Sensible a parámetros                        |
| Ensemble Voting CLasifier (BalancedRandomForest + XGBoost) |Combina fortalezas de ambos modelos, mayor robustez                                 | Más costoso de entrenar                      |



### ¿Qué planteo?

Dependiendo el modelo que eliga, trabajaré con uno u otro target

Clasificación:
Si el objetivo es predecir rating_cat → modelo de clasificación  (Muy Buena, Buena, Regular, Mala)


Regresión:
Si el objetivo es predecir averageRating → modelo de regresión  (escala de 1 a 10 dependiendo la nota )


***

***



**MODIFICACIÓN**   EN BASE A LAS IRRELEVANCIA QUE HAY EN CADA FEATURE, EXISTE UNA NECESIDAD DE MODIFICARLAS PARA DESARROLLAR MEJOR EL DATASET.


1. La columna **runMinute** recibe una modificación Y se incluyen todas las obras por encima de 15 minutos de duración para tener más rango de tipos de obras de trabajo 

2. elimino de la columna **Decade** creada para agrupar los años por décadas, de 1960 hacia abajo para eliminar aquellas décadas con números de votos muy pobres 

![rating_x_decada](graficos_eda/rating_x_decad.png)

### partes del PAIRPLOT más significativas

![pairplot1](graficos_eda/pairplot1.png)



    Relación entre número de votos y la media de nota que recibe la obra audiovisual.Pùedo ver en qué punto se acumulan la mayor cantidad de votos. 



![pairplot2](graficos_eda/pairplot2.png)


    Relaciones varias entre features. Relación bastante definida entre actor_mean_rating con el average rating

### LINEAR REGRESSOR. 

![Linear_reg](graficos_modelo/linear_regr.png)


El modelo no trabaja bien con las features y se aleja mucho de una buena predicción.

### RANDOM FOREST

![RF_reg](graficos_modelo/RF_regres.png)


Este modelo mejora significativamente las métricas 

#### XGBOOST/LIGHTGBM

El **Gradient Boosting** trabaja iterativamente ajustando un nuevo modelo para corregir los errores (residuos) del modelo anterior. Así, cada árbol se ajusta a los residuos de los árboles anteriores, y el modelo global mejora progresivamente.

En el momento de usarlos, me he dado cuenta de que XGBoost y LightGBM han mostrado un desempeño razonablemente bueno, con LightGBM ligeramente superior en términos de R².

Pero aun queda mucho margen de mejora.

### PRUEBAS DE MODELOS DE REGRESIÓN 

| Modelo                             | MSE     | MAE     | RMSE    | R² Score | Observaciones                       |
|-----------------------------------|---------|---------|---------|----------|-------------------------------------|
| Linear Regression / Ridge / Lasso |   1.22      |     0.82    |    1.10     |    0.37     |    El modelo comete errores grandes, equivocandose casi 0.83 puntos de 1  y Solo el 37% de la varianza en los ratings está explicada por el modelo. Esto indica un ajuste pobre                              |
| Random Forest Regressor           | 0.66        |   0.56      |    0.81     |   0.65     |   El modelo se equivoca por 0.57 puntos en la predicción, con una capacidad de generalizar bastante razonable.                                 |
| Gradient Boosting XGBoost  |   0.69  |   0.60      |     0.83    |     0.64     |  El modelo penaliza más los errores grandes, el R Score ligeramente peor. Cn parámetos ajustados puede ser mejor                              || Gradient Boosting XGBoost  |   0.69  |   0.60      |     0.83    |     0.64     |  El modelo penaliza más los errores grandes, el R Score ligeramente peor. Cn parámetos ajustados puede ser mejor                              |
| Gradient Boosting XGBoost **(segundo intento)**  |   0.65  |   0.58      |     0.81    |     0.66     |  Menor error promedio en el descenso de MAE y MSE, No es una mejora drástica, pero si consistente                          |
 Gradient Boosting LightGBM (MLP)              |  0.75       |    0.63     |     0.87    |     0.66     |   Este modelo no da mejores resultados que los anteriores por lo que no parece ser el adecuado para este caso                                  |
| Redes Neuronales              | 0.69        |    0.60     |     0.83    |     0.64     | indica un modelo moderadamente bueno, aunque hay margen de mejora, ya que no capta toda la complejidad del dataset                                | 
| Redes Neuronales  **(segundo intento)**              | 0.71        |    0.62     |     ---    |     0.63    | Usando Bayesian Optimization los resultados no varían significativamente dado la poca complejidad del dataset                             |




### PRUEBA FINAL CON GRADIENT BOOSTING XGBOOST

| Modelo                             | MSE     | MAE     | RMSE    | R² Score | Observaciones                       |
|-----------------------------------|---------|---------|---------|----------|-------------------------------------|
| Gradient Boosting XGBoost  **(INTENTO FINAL)**              |    0.63     |    0.56     |    0.79    |  0.67    |  Mejor desempeño general entre los modelos probados. Explica el 67% de la varianza de los datos, con un error absoluto promedio menor a 0.6. Muestra solidez sin sobreajuste evidente. Optimizado con hiperparámetros ajustados.  

![comparacion__modelos](graficos_modelo/comparcion_regresion.png)

Este gráfico compara visualmente el desempeño de distintos modelos de regresión mediante tres métricas clave: MSE, MAE y R². Se observa que el modelo Gradient Boosting (Final) obtiene los valores más bajos de error (MSE y MAE), lo que indica mayor precisión. Además, tiene el R² más alto (0.672), mostrando que explica mejor la varianza de los datos. Modelos como la regresión lineal muestran peores resultados, con errores más altos y menor capacidad predictiva. En conjunto, el gráfico evidencia que Gradient Boosting Final es el modelo más robusto.

***
***

### CONTEXTO ANTES DE EMPEZAR CLASIFICACIÓN 

![CONTEO_rating](graficos_eda/count_rating.png)

Como se puede observar, se muestra claramente una distribución desequilibrada  con la categoría "buena" siendo con diferencia la más frecuente (~155,000 registros).
Esto representa un riesgo de sesgo para muchos modelos, que tienden a optimizar para la clase mayoritaria.  


Dado el problema de desbalance de clases Estudié diferentes estrategias para mitigar:

1. Modelos estándar como línea base Logistic Regression y Random Forest sirvieron para evaluar qué tan grave era el problema con modelos sin ajustes.
Resultado: pobre rendimiento en clases minoritarias.

2. Random Forest con hiperparámetros
Busqué mejorar con optimización, pero no resolvió el fondo del problema (seguía el sesgo hacia clases frecuentes).

3. XGBoost con y sin pesos
XGBoost ofrece buen rendimiento por diseño, pero no soluciona el desbalance por sí solo. Mejoré algo el recall de clases minoritarias ("mala", "muy buena").

4. Modelos balanceados (Balanced RF, LightGBM con pesos)
Estas versiones modifican internamente la forma en que se entrenan los árboles para prestar más atención a clases menos representadas.

Resultado: mejor cobertura (recall alto), pero con pérdida de precisión (más falsos positivos).

5. Modelo final: VotingClassifier (XGBoost + Balanced RF) XGBoost aporta precisión y Balanced RF recall.

Resultado: un equilibrio robusto, especialmente importante cuando hay interés en detectar correctamente las clases raras.  No es una solución mágica, pero estable.

### PRUEBAS DE MODELOS DE CLASIFICACIÓN


| Modelo                             | Accuracy     | Precision     | Recall   | F1-Score | Observaciones                       |
|-----------------------------------|---------|---------|---------|----------|-------------------------------------|
| Logistic Regression (multiclase)  |         0.70                 | B.0.77   M.0.59  MB.0.50  R.0.56        |   B.0.87    M.0.27   MB.0.00   R.0.53      |  B.0.81  M.0.37  MB.0.00  R.0.54   | Primera toma de contacto. El modelo tiene buen rendimiento con la clase 'BUENA' (f1=0.81), pero falla completamente con 'MUY BUENA' y 'MALA' (recall 0.27). no está capturando bien las clases minoritarias.
| Random Forest Classifier   |     0.76    |   B.0.86   M.0.57  MB.0.60  R.0.63   |  B.0.87    M.0.40   MB.0.30   R.0.68  |  B.0.86   M.0.47   MB.0.40   R.0.65  | Equilibrio moderado entre clases, pero aún hay margen de mejora, especialmente en clases minoritarias. Las clases Mala y Muy buena tienen baja recall, especialmente en "muy buena" (0.30), lo que indica que el modelo no está identificando bien estas clases.
| Random Forest Classifier **Con hiperparámetros**   |     0.77   |   B.0.86   M.0.57  MB.0.60  R.0.63   |  B.0.87   M.0.40   MB.0.30   R.0.69  |  B.0.86  M.0.47   MB.0.40  R.0.66  | Casi no hay diferencia con o sin parametrización. Llegando al límiteóptimo del modelo.
| XGBoost  Classifier| 0.77 | B.0.86   M.0.60  MB.0.66  R.0.63  |  B.0.87   M.0.41  MB.0.27  R.0.70 |  B.0.87   M.0.48  MB.0.38  R.0.66 |  XGBClassifier espera etiquetas numéricas- Uso **LabelEncoder** para trabajar. |
 XGBoost  Classifier **pesos ajustados x clase** |  0.74 | B.0.90   M.0.46  MB.0.33  R.0.60   | B.0.80   M.0.58  MB.0.54  R.0.69   |  B.0.85   M.0.51  MB.0.41  R.0.64  |    El accuracy y la precisión general bajan ligeramente al aplicar pesos,  ligera pérdida de precisión en 'BUENA', pero mejora notable en recall para clases 'MUY BUENA' y 'MALA' |
/ LightGBM Classifier **pesos ajustados**     |        0.65                     |   B.0.91   M.0.43  MB.0.11  R.0.58                      |B.0.65   M.0.71  MB.0.77  R.0.64 |  B.0.76   M.0.53  MB.0.20  R.0.61   | lase 2 tiene recall 0.77 pero precision 0.11 indicando muchos falsos positivos ya El modelo está clasificando demasiadas instancias como clase 'MALA', lo que daña la precisión general.Los pesos hacen que LightGBM “se obsesione” con detectar clases raras.
|BalancedRandomForestClassifier | 0.66  | B.0.91   M.0.43  MB.0.13  R.0.58 |B.0.67   M.0.71  MB.0.77  R.0.63 | B.0.77   M.0.53  MB.0.22  R.0.60   |En general, el modelo aprende a identificar las clases más difíciles, aunque sacrifica precisión (muchos falsos positivos).

#### OPCIÓN FINAL DE PRUEBA: Ensemble: votingClassifier (XGBoost + Balanced RF)

| Modelo                             |  Accuracy    | Precisión    | Recall   | f1-Score | Observaciones                       |
|-----------------------------------|---------|---------|---------|----------|-------------------------------------|
|**Híbrido** votingClassifier (XGBoost + Balanced RF) |     0.77   |   B.0.88   M.0.57  MB.0.57  R.0.63    |  B.0.85   M.0.47  MB.0.35  R.0.71   |   B.0.87   M.0.51  MB.0.43  R.0.67  | Mantiene la precisión de XGBoost, especialmente para las clases dominantes y Gana algo en recall para clases difíciles gracias al efecto del Balanced Random Forest.  

![F1-Score](graficos_modelo/f1-Score_clas.png)

Mejor F1 para clases minoritarias (MALA y MUY BUENA) respecto a otros modelos, que las ignoraban casi completamente. El ensemble combina fuerzas: XGBoost aporta precisión, Balanced RF mejora recall en clases minoritarias.


![Matriz](graficos_modelo/MC_hibrido_clasif.png)

Buen desempeño general, aunque demasiadas BUENAS están siendo etiquetadas como REGULARES, lo que puede desvirtuar la calidad del output.

El mayor punto débil del modelo es que la mayoría de las MUY BUENAS están siendo etiquetadas como simplemente BUENAS. No sabe diferenciar suficientemente entre MUY BUENA y BUENA. Quizás haya solapamiento fuerte entre estas.El modelo confunde MUY BUENA con BUENA y REGULAR con BUENA o MALA que no dejan de ser clases cercanas o con características similares.

Esto es común en problemas con clases muy similares o con solapamiento en las features.

Pero igualemente, a pesar de confusiones internas, mantiene buena precisión y recall globales.

___

Un detalle interesante es empezar a **trabajar con pesos** para darle menor valor a las clases mayoritarias y cederle espacio a las minoritarias. Y elige según tu prioridad:

+ Si Prefiero detectar bien todas las clases, aunque baje un poco el accuracy usando pesos en el modelo XGBoost veo esos resultados. 

- Si prefiero máxima exactitud global sin importa fallar en las clases pequeñas, prescindo de los pesos.

F1 Macro: 0.60 en promedio, el modelo tiene un desempeño moderado en todas las clases ya que no ignora las minoritarias. 

F1 Weighted: 0.75 como la clase 0 es muy grande, su buen rendimiento arrastra hacia arriba el promedio, aunque el modelo falle más en las clases pequeñas.  Favoreciendo las clases grandes.

        A DESTACAR: no entreno ni con KNN ni con SVM ya que estos métdos no funcionan bien con alta demanda de datos y no trabajan bien con un desvalance importante de clases que es mi caso

#### Grafícos de las métricas de los diferentes modelos 

![Accuracy](graficos_modelo/accuracy_class.png)

![Precision](graficos_modelo/precision_class.png)

![Recall](graficos_modelo/recall_class.png)

![F1Score](graficos_modelo/fiscore_class.png)

***

***



### CONCLUSIÓN 

Viendo el dataset que se me presenta, abordé la búsqueda de mi objetivo desde el traget original usando modelos de regressión, y creando una nueva columna categórica para abordarlo desde modelos de clasificación.

El escenario que se me presentaba era muy desvalanceado pero me enfoqué en sacarle el mayor partido

Durante la primera fase, se abordó el problema como una **regresión multivariable**, donde el objetivo era predecir directamente el valor numérico del rating: Se probaron modelos como Linear Regression, Random Forest Regressor, XGBoost Regressor y LightGBM.

Aunque algunos modelos lograron buen desempeño en términos de RMSE y R², se observó:

+ Alta variabilidad en el error según el segmento.

+ Dificultad para predecir valores extremos con precisión (sobre todo ratings muy altos o muy bajos).

Como no conseguí un gran resulado, cambié el enfoque del problema como uno de clasificación por categorías.

**Modelos de Clasificación** (Categorías de rating)
En la segunda fase, el rating se transforma en una variable categórica: "muy buena", "buena", "regular", "mala", permitiendo aplicar técnicas más interpretables y robustas frente al desequilibrio de clases.
Usé  múltiples modelos: Logistic Regression, Random Forest, XGBoost, LightGBM, y modelos balanceados (Balanced RF, pesos por clase).

El principal desafío fue el grave desbalance de clases:

+ "Buena" dominaba el dataset.

+ "Muy buena" y "mala" eran escasas, dificultando su identificación.


¿Qué Estrategias apliqué para abordar el desbalance?

1. Pesos ajustados por clase (XGBoost, LightGBM).

2. Implementación de modelos balanceados como BalancedRandomForestClassifier.


**Evaluación con métricas por clase:** precision, recall y F1-score.

**Modelo final elegido:**
***VotingClassifier (XGBoost + BalancedRandomForest)***

Combinó lo mejor de ambos mundos:

+ XGBoost → Alta precisión en clases frecuentes.

- Balanced RF → Mayor sensibilidad (recall) en clases minoritarias.

Logró un F1-score equilibrado entre todas las clases, sin sacrificar demasiado accuracy global.



Enfoqué el trabajo como un caso que pudiera tocarme fácilmente en un caso real como la existencia de desvalance y falta de datos.  Por tanto, me orienté  a resolver problemas lo mejor posible. El modelo teóricamente correcto (regresión) no me sirvió y pasé a clasificación, respaldado por pruebas empíricas y métricas claras.
